In [ ]:
import gc
import sys
import shutil
from glob import glob
from pathlib import Path
from tempfile import TemporaryDirectory
from PIL import Image
import numpy as np
import torch
from datasets import load_dataset, IterableDataset
from humanize import naturalsize
from tqdm import tqdm
from loguru import logger
from transformers import pipeline
from itertools import batched


In [2]:
# easy timestamps
logger.remove()
logger.add(sys.stdout, level="INFO")

1

### 1. Set variables for test

In [3]:
BATCH_SIZE = 4
MODEL_NAME = "timm/vit_small_patch14_reg4_dinov2.lvd142m"
TEST_DATASET = "kvriza8/microscopy_images"
NUM_TEST_IMAGES = 256

## 2. Define way to download small test dataset 

In [4]:
def dl_hf_images(dataset_name: str = "kvriza8/microscopy_images",
                 dir: Path = None,
                 max_images: int = 64,
                 overwrite: bool = True,
                 format: str = "png") -> None:

    dataset = load_dataset(dataset_name, split="train", streaming=True)
    if overwrite:
        shutil.rmtree(dir, ignore_errors=True)
        dir.mkdir(parents=True, exist_ok=True)

    image_paths = []
    for i, img_row in enumerate(tqdm(iter(dataset), total=max_images)):
        if i >= max_images:
            break
        img = img_row["image"]
        image_paths += [(dir / f"{i}.{format}")]
        img.save(image_paths[-1])

    print(f"Size of images on disk: {naturalsize(sum([p.stat().st_size for p in image_paths]))}")

    del dataset
    gc.collect()

    return None

In [5]:
%load_ext memray

In [ ]:
def stream_images(pathlist):
    for p in pathlist:
        yield Image.open(p.as_posix()).convert("RGB").copy()



In [7]:
def images_from_paths(pathlist):
    return (Image.open(p.as_posix()).convert("RGB").copy() for p in pathlist)

In [11]:
with TemporaryDirectory() as tmp:    
    logger.info("Downloading test images.")
    dl_hf_images(dir=Path(tmp), max_images=NUM_TEST_IMAGES)
    imagepaths = list(Path(tmp).glob("*.png"))
    gen = images_from_paths(imagepaths)
    for i in range(1):
        im = next(gen)
        im.show()

2025-03-29 21:25:16.208 | INFO     | __main__:<module>:2 - Downloading test images.


100%|██████████| 256/256 [00:16<00:00, 15.36it/s]


Size of images on disk: 19.5 MB


In [ ]:
%%memray_flamegraph --native --follow-fork --temporal

with TemporaryDirectory() as tmp:
    logger.info("Downloading test images.")
    dl_hf_images(dir=Path(tmp), max_images=NUM_TEST_IMAGES)
    imagepaths = list(Path(tmp).glob("*.png"))
    embeddings = []

    # ds = IterableDataset.from_generator(lambda: stream_images(imagepaths))

    logger.info("Instantiating pipeline.")
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pipe = pipeline(task="image-feature-extraction",
                    model=MODEL_NAME, device=DEVICE, pool=True, use_fast=True)
            

    logger.info("Starting embedding pipeline.")

    for out in tqdm(pipe(images_from_paths(imagepaths), batch_size=BATCH_SIZE),
                    total=len(imagepaths)//BATCH_SIZE):
        embeddings += out

    logger.info("Done with embedding pipeline.")

2025-03-29 21:11:47.491 | INFO     | __main__:<module>:11 - Downloading test images.


Memray WARNING: Correcting symbol for malloc from 0x420620 to 0x7f334b469c60
Memray WARNING: Correcting symbol for free from 0x420ab0 to 0x7f334b46a370
100%|██████████| 256/256 [00:17<00:00, 14.42it/s]


Size of images on disk: 19.5 MB
2025-03-29 21:12:07.673 | INFO     | __main__:<module>:18 - Instantiating pipeline.


Device set to use cpu


2025-03-29 21:12:10.518 | INFO     | __main__:<module>:24 - Starting embedding pipeline.


...

2025-03-29 21:15:17.472 | INFO     | __main__:<module>:30 - Done with embedding pipeline.


Output()

⚠  No debug information was found for the Python interpreter  ⚠

Without debug information reports showing native traces may not include file names and line numbers. Please use an 
interpreter built with debug symbols for best results. Check https://bloomberg.github.io/memray/native_mode.html 
for more information regarding how memray resolves symbols.

Output()

Results saved to /home/willsa/git/bedmap-dev/nbs/daft-try/memray-results/tmp482mgjyc/flamegraph.html